### 0. Notebook Setup

##### Importing Appropriate Packages

In [1]:
# General Packages

import numpy as np
import pickle
import os
import sys
import json


# Torch Specific

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from torchvision import models,transforms,datasets

from PIL import Image


# Project Specific

sys.path.append(str(os.getcwd() + '/scripts'))
from Preprocessing import MultiLabelDataset

###### Constants and Parameters

In [2]:
BATCH_SIZE = 16
IS_SHUFFLE = True
NUM_WORKERS = 0

LEARNING_RATE = 0.001

###### Machine Setup

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


### 1. Loading Data

##### Importing Data

In [4]:
# Loading the tupple of tensors from the pickle file
with open('data/train_data.pickle', 'rb') as handle:
    train = pickle.load(handle)

###### Data Loader

In [5]:
# Building the data loader from the tupple of tensors
train_loader = DataLoader(train,
                          batch_size=BATCH_SIZE,
                          shuffle=IS_SHUFFLE,
                          num_workers=NUM_WORKERS)

### 2. Building Model

##### Transfer Learning

In [62]:
# Loading the pretrained VGG model
model_vgg = models.vgg16(pretrained=True)

# Freezing all layers
for param in model_vgg.parameters():
    param.requires_grad = False

# Adding linear layer
model_vgg.classifier._modules['6'] = torch.nn.Linear(4096, 13)
model_vgg.classifier._modules['7'] = torch.nn.Sigmoid()

# Defining the optimizer
optimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = LEARNING_RATE)

In [64]:
def train_model(model, dataloader, size, epochs=1, optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in dataloader:
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            loss = torch.nn.BCELoss()
            loss = loss(outputs, classes)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [65]:
train_model(model_vgg, 
            train_loader,
            size=len(train),
            epochs=2,
            optimizer=optimizer_vgg)

RuntimeError: The size of tensor a (16) must match the size of tensor b (13) at non-singleton dimension 1

In [46]:
for image, classnames in train_loader:
    target = torch.tensor([[0,1,0,-2,0,0,0,0,0,0,0,0,0]], dtype=torch.float32)
    loss = torch.nn.BCELoss()
    loss = loss(target, classnames[0])
    print(loss)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([1, 13])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got -2.000000 at ../aten/src/THNN/generic/BCECriterion.c:60

In [34]:
for image, classnames in train_loader:
    print(classnames[0])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 